In [11]:
import sys
import json
import requests
import time
import argparse
import pandas as pd
import numpy as np
# import pymongo

import random

import boto3
import boto.s3.connection
from boto.s3.key import Key
from src.Airbnb_Scraping_tools import *
from urllib.parse import urlparse
from urllib.request import urlretrieve 
import os

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load in data

In [13]:
s3_path = 'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbData/'

In [14]:
s3_path

'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbData/'

In [15]:
folder = 'Seattle-washington-US'
file_tmp = pd.read_csv(s3_path+folder+'/listings.csv.gz',compression = 'gzip')
print('Total number of apartments in ' + folder +" is: {}".format(len(file_tmp)))
file_tmp.describe()

Total number of apartments in Seattle-washington-US is: 31253


/home/ubuntu/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,scrape_id,host_id,host_acceptance_rate,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month
count,3.125300e+04,3.125300e+04,3.125300e+04,0.0,31239.000000,31239.000000,0.0,31253.000000,31253.000000,31253.000000,...,31253.000000,23726.000000,23689.000000,23688.000000,23661.000000,23690.000000,23660.000000,23648.000000,31253.000000,24030.000000
mean,1.085829e+07,2.017050e+13,3.632524e+07,NaN,5.171132,5.171132,NaN,34.056220,-118.334252,3.425751,...,20.842543,94.214533,9.612394,9.416456,9.770297,9.768974,9.612933,9.485411,3.546572,1.857360
std,5.527237e+06,3.531306e+00,3.454658e+07,NaN,24.143834,24.143834,NaN,0.101912,0.153783,2.481926,...,38.106641,8.040304,0.789394,0.954680,0.652473,0.658068,0.735289,0.834739,5.551354,2.022831
min,1.090000e+02,2.017050e+13,5.900000e+01,NaN,0.000000,0.000000,NaN,33.338905,-118.911026,1.000000,...,0.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.020000
25%,6.262191e+06,2.017050e+13,7.441819e+06,NaN,1.000000,1.000000,NaN,34.013997,-118.431313,2.000000,...,1.000000,92.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,1.000000,0.410000
50%,1.216199e+07,2.017050e+13,2.471631e+07,NaN,2.000000,2.000000,NaN,34.066607,-118.349465,2.000000,...,6.000000,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000,1.120000
75%,1.570678e+07,2.017050e+13,5.623875e+07,NaN,4.000000,4.000000,NaN,34.103960,-118.267693,4.000000,...,23.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,3.000000,2.730000
max,1.852404e+07,2.017050e+13,1.285513e+08,NaN,855.000000,855.000000,NaN,34.742088,-117.655471,16.000000,...,605.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,49.000000,90.000000


# Open S3 bucket

In [16]:
bucket_name = "chen-gal-test"

In [17]:
s3 = boto3.client("s3")

remote_pathname_l1 = 'AirbnbData/'
remote_pathname_l2 = folder
remote_pathname = remote_pathname_l1 + remote_pathname_l2

# s3.put_object(Body=file , Bucket=bucket_name, Key=remote_pathname)


In [18]:
remote_pathname

'AirbnbData/Seattle-washington-US'

In [20]:
logfile_name = 'DataSet/'+'logfile_'+folder+'.txt'
print(f'Logfile name is {logfile_name}')

Logfile name is DataSet/logfile_Seattle-washington-US.txt


In [21]:
if not os.path.exists('DataSet/'):
    os.makedirs('DataSet/')

# Downloading

In [22]:

apt_infos = {}
apt_jses = {}

In [ ]:
for i in range(0,len(file_tmp)):
# for i in range(1):
    print(i)
    url = file_tmp.loc[i,'listing_url']
    apt_id = file_tmp.loc[i,'id']
    apt_name = file_tmp.loc[i,'name']
    with open(logfile_name,'a') as f:
        f.write(f'---------------------\nApartment{i} | {apt_id} | {url} \n')         
    try:
        return_parsed = parse_each_webpage(url)
    except CannotOpenUrl:        
        with open(logfile_name,'a') as f:
            f.writelines('Cannot open '+ str(i)+ ' url.\t' + 'Id: '+ str(apt_id) + ' |  Url: '+ url)                        
        print('Failed to get access to URL, sleep for 20 secs ')
        time.sleep(20)
        return_parsed = parse_each_webpage(url)
    finally:
        scrape_info,ori_json = return_parsed[0],return_parsed[1]
        if len(scrape_info) == 0:
            with open(logfile_name,'a') as f:
                f.writelines('Failed to scrape the url, simply save the json file\n')
        else:
            with open(logfile_name,'a') as f:
                f.writelines('Successfully scraped the url, save both files\n')
            
        dict_apt = {'apt_name':apt_name,'url' : url, 'info': scrape_info}
#         apt_jses.update({str(apt_id):ori_json})
        apt_infos.update({str(apt_id):dict_apt})
        with open(logfile_name,'a') as f:
                f.writelines(f'Finished scraping {i} apartment! ID:{apt_id}\n')
                
#         if i % 75 == 0 and i != 0:            
#             js_filename = remote_pathname +"/jsons/ori_json_"+str(i)            
#             s3.put_object(Body=json.dumps(apt_jses) , Bucket=bucket_name, Key=js_filename)
            
#             print("Wrote  down"+str(i)+"json file")
#             with open(logfile_name,'a') as f:
#                 f.writelines(f'Saved json file {js_filename}!\n') 
        if i % 2000 == 0 and i != 0:            
            wsdata_filename = remote_pathname +"/ws_data/scraped_partial_data_"+str(i)            
            s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=wsdata_filename)
            
            print("Wrote  down"+str(i)+"ws_data")
            with open(logfile_name,'a') as f:
                f.writelines(f'Saved ws_data file {wsdata_filename}!\n')  
            apt_infos = {}
            
        time.sleep(random.random()+1)
        
#upload log_file to s3
logfile_s3_filename = remote_pathname+'/ws_data/log_file.txt'
s3.upload_file(Bucket = bucket_name,Filename=logfile_name,Key=logfile_s3_filename)

scraped_filename = remote_pathname +"/ws_data/webscrapted_last_piece.json"

s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=scraped_filename)

0
request_url:https://www.airbnb.com/rooms/16228948
1
request_url:https://www.airbnb.com/rooms/8909380
2
request_url:https://www.airbnb.com/rooms/14078522
3
request_url:https://www.airbnb.com/rooms/13006928
4
request_url:https://www.airbnb.com/rooms/7898757
5
request_url:https://www.airbnb.com/rooms/11254337
6
request_url:https://www.airbnb.com/rooms/16068508
7
request_url:https://www.airbnb.com/rooms/654641
8
request_url:https://www.airbnb.com/rooms/14859779
9
request_url:https://www.airbnb.com/rooms/4839375
10
request_url:https://www.airbnb.com/rooms/14281484
11
request_url:https://www.airbnb.com/rooms/8428885
12
request_url:https://www.airbnb.com/rooms/1557529
13
request_url:https://www.airbnb.com/rooms/18042820
14
request_url:https://www.airbnb.com/rooms/5393269
15
request_url:https://www.airbnb.com/rooms/5851860
16
request_url:https://www.airbnb.com/rooms/17144596
17
request_url:https://www.airbnb.com/rooms/6153123
18
request_url:https://www.airbnb.com/rooms/13748272
19
request_ur

HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

114
request_url:https://www.airbnb.com/rooms/6201466
115
request_url:https://www.airbnb.com/rooms/892469
116
request_url:https://www.airbnb.com/rooms/8012663
117
request_url:https://www.airbnb.com/rooms/5805878
118
request_url:https://www.airbnb.com/rooms/13605429
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

119
request_url:https://www.airbnb.com/rooms/17865517
HTTP error: 503 Service Temporarily Unavailable
<!DOCTYPE html>

<html>
<head>
  <meta charset="utf-8">
  <title>503 Service Unavailable - Airbnb</title>

  <link href="https://a0.musca

173
request_url:https://www.airbnb.com/rooms/9920792
HTTP error: 503 Service Temporarily Unavailable
<!DOCTYPE html>

<html>
<head>
  <meta charset="utf-8">
  <title>503 Service Unavailable - Airbnb</title>

  <link href="https://a0.muscache.com/airbnb/static/packages/common_o2.1-39a35b3574fe2cc79038c20fabd3669c.css" media="all" rel="stylesheet" type="text/css">

  <link rel="shortcut icon" type="image/x-icon" href="https://a1.muscache.com/airbnb/static/logotype_favicon_pretzel-114df7f43fae7dd6dbc4ab074d934da5.ico">

  <!-- Replace by loading from stylesheet -->
  <style type="text/css">
    .text-ginormous {
      font-size: 85px;
    }
  </style>
</head>

<body>
  <header class="airbnb-header">
    <a href="/" class="header-belo"></a>
  </header>
  <div class="page-container-responsive space-top-8 space-8">
    <div class="row space-6">
      <div class="col-md-6">
        <h1 class="text-jumbo text-ginormous">Stay tuned</h1>
        <h6>Error code: 503</h6>

        <p>
          Ai

HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

212
request_url:https://www.airbnb.com/rooms/2387536
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>

219
request_url:https://www.airbnb.com/rooms/4090410
220
request_url:https://www.airbnb.com/rooms/3413115
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

221
request_url:https://www.airbnb.com/rooms/1914160
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

222
request_url:https://www.airbnb.com/rooms/12918311
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

HTTP error: 503 Service Temporarily Unavail

...

HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

692
request_url:https://www.airbnb.com/rooms/13929036
693
request_url:https://www.airbnb.com/rooms/11637944
HTTP error: 503 Service Temporarily Unavailable
<!DOCTYPE html>

<html>
<head>
  <meta charset="utf-8">
  <title>503 Service Unavailable - Airbnb</title>

  <link href="https://a0.muscache.com/airbnb/static/packages/common_o2.1-39a35b3574fe2cc79038c20fabd3669c.css" media="all" rel="stylesheet" type="text/css">

  <link rel="shortcut icon" type="image/x-icon" href="https://a1.muscache.com/airbnb/static/logotype_favicon_pretzel-114df7f43fae7dd6dbc4ab074d934da5.ico">

  <!-- Replace by loading from stylesheet -->
  <style type="text/css">
    .text-ginormous {
      font-size: 85px;
    }
  </style>
</head>

<body>
  <header class="airbnb-

707
request_url:https://www.airbnb.com/rooms/17075450
708
request_url:https://www.airbnb.com/rooms/16239355
HTTP error: 503 Service Temporarily Unavailable
<!DOCTYPE html>

<html>
<head>
  <meta charset="utf-8">
  <title>503 Service Unavailable - Airbnb</title>

  <link href="https://a0.muscache.com/airbnb/static/packages/common_o2.1-39a35b3574fe2cc79038c20fabd3669c.css" media="all" rel="stylesheet" type="text/css">

  <link rel="shortcut icon" type="image/x-icon" href="https://a1.muscache.com/airbnb/static/logotype_favicon_pretzel-114df7f43fae7dd6dbc4ab074d934da5.ico">

  <!-- Replace by loading from stylesheet -->
  <style type="text/css">
    .text-ginormous {
      font-size: 85px;
    }
  </style>
</head>

<body>
  <header class="airbnb-header">
    <a href="/" class="header-belo"></a>
  </header>
  <div class="page-container-responsive space-top-8 space-8">
    <div class="row space-6">
      <div class="col-md-6">
        <h1 class="text-jumbo text-ginormous">Stay tuned</h1>
   

714
request_url:https://www.airbnb.com/rooms/17220879
715
request_url:https://www.airbnb.com/rooms/18041797
716
request_url:https://www.airbnb.com/rooms/9771001
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

717
request_url:https://www.airbnb.com/rooms/16630334
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

718
request_url:https://www.airbnb.com/rooms/15860683
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</bo

HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

724
request_url:https://www.airbnb.com/rooms/14154925
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

725
request_url:https://www.airbnb.com/rooms/14810362
726
request_url:https://www.airbnb.com/rooms/9961396
727
request_url:https://www.airbnb.com/rooms/17759908
728
request_url:https://www.airbnb.com/ro

HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

738
request_url:https://www.airbnb.com/rooms/2547335
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center><h1>503 Service Temporarily Unavailable</h1></center>
<hr><center>nginx</center>
</body>
</html>

739
request_url:https://www.airbnb.com/rooms/13883680
HTTP error: 503 Service Temporarily Unavailable
<html>
<head><title>503 Service Temporarily Unavailable</title></head>
<body bgcolor="white">
<center

In [15]:
logfile_s3_filename = remote_pathname+'/ws_data/log_file.txt'
s3.upload_file(Bucket = bucket_name,Filename=logfile_name,Key=logfile_s3_filename)

In [102]:
with open(logfile_name,'a') as f:
    f.writelines(f'Saved json file {js_filename}!\n')                            
jses = {}

## Save the result

### save scrapted data

In [16]:
scraped_filename = remote_pathname +"/ws_data/webscrapted.json"

s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=scraped_filename)

{'ETag': '"e5be8054bac84d9f037ecc369719eb7d"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Wed, 02 May 2018 04:27:30 GMT',
   'etag': '"e5be8054bac84d9f037ecc369719eb7d"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'rjbHFiw1U3qRRNSXOpal38U9mq2s7b7uRxZNKlUeZvrbBGwWPBGXSUeC4HBQ13A30POoEh8E8c8=',
   'x-amz-request-id': 'FA72BE06AE13EEC8'},
  'HTTPStatusCode': 200,
  'HostId': 'rjbHFiw1U3qRRNSXOpal38U9mq2s7b7uRxZNKlUeZvrbBGwWPBGXSUeC4HBQ13A30POoEh8E8c8=',
  'RequestId': 'FA72BE06AE13EEC8',
  'RetryAttempts': 0}}

### save json data

In [88]:
# last_piece_js_filename = remote_pathname+"/jsons/ori_json_lastpiece.json"

# s3.put_object(Body=json.dumps(apt_jses) , Bucket=bucket_name, Key=last_piece_js_filename)


{'ETag': '"7b336feb550bfad1dc2c55de6ec8a93d"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Mon, 30 Apr 2018 01:22:39 GMT',
   'etag': '"7b336feb550bfad1dc2c55de6ec8a93d"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'S7BG7n3tOOJQXdwejYwoPOwpqzfEF9NGChTaAzo/bBZ5ZmmQJvaioLk2y3mYnDQtAsnUZ52XF/k=',
   'x-amz-request-id': 'BB474E35A45B7DAC'},
  'HTTPStatusCode': 200,
  'HostId': 'S7BG7n3tOOJQXdwejYwoPOwpqzfEF9NGChTaAzo/bBZ5ZmmQJvaioLk2y3mYnDQtAsnUZ52XF/k=',
  'RequestId': 'BB474E35A45B7DAC',
  'RetryAttempts': 0}}

## Load json data from s3

In [11]:
scraped_filename = 'AirbnbData/Boston-Massachusetts-US/ws_data/scraped_partial_data_3600'

response = s3.get_object(Bucket=bucket_name,
                         Key=scraped_filename)

In [19]:
response

{'AcceptRanges': 'bytes',
 'Body': <botocore.response.StreamingBody at 0x7fee21cc0d30>,
 'ContentLength': 5059688,
 'ContentType': 'binary/octet-stream',
 'ETag': '"3aa75aea13d6b1d083cea56b1332defc"',
 'LastModified': datetime.datetime(2018, 4, 30, 14, 25, 10, tzinfo=tzutc()),
 'Metadata': {},
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '5059688',
   'content-type': 'binary/octet-stream',
   'date': 'Tue, 01 May 2018 05:46:32 GMT',
   'etag': '"3aa75aea13d6b1d083cea56b1332defc"',
   'last-modified': 'Mon, 30 Apr 2018 14:25:10 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'VgQXishVBTFtclbB8DJ3hcIxNKV9hLHaJQwz+0jizaRHyz3WHejmgO/NCmR2NeXNjU4CMeGdTJo=',
   'x-amz-request-id': 'D35DCF8A8B8701D1'},
  'HTTPStatusCode': 200,
  'HostId': 'VgQXishVBTFtclbB8DJ3hcIxNKV9hLHaJQwz+0jizaRHyz3WHejmgO/NCmR2NeXNjU4CMeGdTJo=',
  'RequestId': 'D35DCF8A8B8701D1',
  'RetryAttempts': 0}}

In [12]:
scraped_data = json.loads(response['Body'].read())

In [13]:
apt_infos = scraped_data

In [14]:
len(apt_infos)

3600